In [1]:
import pandas as pd
import numpy as np
from pprint import pprint

df = pd.read_csv('pericias_medicas.csv', sep=',',  encoding='utf-8')
#convierte en string los datos de la columna text
df1 = df['text'].apply(str)

In [5]:
import re
def general(txt: str, bert=False, nums=False) -> str:
    """
    Elimina caracteres no deseados
    Params:
        **txt**:texto a ser limpiado de caracteres no desaeados
    """
    if nums:
        txt = re.sub(r'\d+', ' ', txt)
    if not bert:
        txt = txt.translate(str.maketrans(
            'áéíóúýàèìòùÁÉÍÓÚÀÈÌÒÙÝ', 'aeiouyaeiouAEIOUAEIOUY'))
        txt = re.sub(r'[^\w\s:.)-]', '', txt)

    txt = txt.replace('\r', ' ').replace("\v", ' ').replace(
        "\t", ' ').replace("\f", ' ').replace("\a", ' ').replace("\b", ' ')
    txt = re.sub(' +', ' ', txt)
    txt = txt.strip()
    return txt

dfLimpio = df1.apply(general)

In [ ]:
# Segunda fase de limpieza
# Se eliminan todos los elementos que meten ruido al texto y que no fueron eliminados en la fase de limpieza 1.
import re

def limpiarTexto2(text):
    text = re.sub('\d{2,10}','',text)
    text = re.sub('\d\n','',text)
    text = re.sub(' +\n','\n',text)
    text = re.sub('\n+','\n',text)
    text = re.sub('\n',' \n',text)
    text = re.sub('\d \n','',text)
    text = re.sub('\x0c','',text)
    text = re.sub('\u200b\n','',text)
    text = re.sub(r'[nN]º|[nN][. ]º','',text)
    text = re.sub('[a-zA-Z]+.com','',text)
    #text = re.sub('[\.)-]+[\s]{0,3}','-',text) #PRUEBA PARA NORMALIZAR NRO ROMANO.
    return text

dfPrueba = dfLimpio.apply(limpiarTexto2)

In [ ]:
import nltk
import re

stops = nltk.corpus.stopwords.words('spanish')

def remove_stops(texto: str) -> str:
    """
    Función que elimina stopwords
    Params:
        **texto**:texto a ser limpiado de stopwords

    """
    texto = [
        i for i in texto.split() if i not in stops
    ]
    return ' '.join(texto)

dfLimpio = dfLimpio.apply(remove_stops)

AttributeError: 'list' object has no attribute 'upper'

In [ ]:
# Busca títulos en mayusculas
def buscarTitulosMayusculas(text):
    # Expresiín regular para encontrar títulos en mayusculas.
    tituloMayusculas =re.compile(r'(I{1,3}|IV|V|VI{1,3}|IX|X)[.) -]+[A-Z -]+[:\.\n]') #[1-9]|
    titulosMayusculasEncontrados = []

    for m in tituloMayusculas.finditer(text):
        if len(m.group()) > 8:
            titulosMayusculasEncontrados.append(m.group())

    return titulosMayusculasEncontrados
# titulosMAyuscula: lista que guarda los títulos en mayusculas
titulosMayusculas=[]
for expediente in dfPrueba:
    titulosMayusculas.append(buscarTitulosMayusculas(expediente))

dfTitulosMayusculasConStops = pd.DataFrame(titulosMayusculas)

In [ ]:
# Busca títulos en mayusculas
def buscarTitulosLatinosMayusculas(text):
    # Expresiín regular para encontrar títulos en mayusculas.
    tituloMayusculas =re.compile(r'[0-9][.) -]+[A-Z -]+[:\.\n]') #[1-9]|
    titulosMayusculasEncontrados = []
    i=1
    for m in tituloMayusculas.finditer(text):
        if len(m.group()) > 8 and re.findall(str(i),m.group()):
            titulosMayusculasEncontrados.append(m.group())
            i+=1

    return titulosMayusculasEncontrados
# titulosMAyuscula: lista que guarda los títulos en mayusculas
titulosLatinosMayusculas=[]
for expediente in dfPrueba:
    if len(buscarTitulosLatinosMayusculas(expediente)) > 3:
        titulosLatinosMayusculas.append(buscarTitulosLatinosMayusculas(expediente))

In [ ]:
pprint(titulosMayusculas)

In [ ]:
def crearDic(dic):
    dic = re.sub(r'(I{1,3}|IV|V|VI{1,3}|IX|X)[-.) ]+','',dic)
    dic = re.sub(r'[^\w\s -]','',dic)
    dic = re.sub(r'-',' ',dic)
    dic = dic.lower()
    dic = remove_stops(str(dic))
    dic = dic.upper()
    dic = dic.split()
    return dic

dicTitulos = crearDic(str(titulosMayusculas))

In [ ]:
# Busca títulos en mayusculas
def buscarTitulosLatinosMayusculas(text, dic):
    # Expresiín regular para encontrar títulos en mayusculas.
    tituloMayusculas =re.compile(r'[0-9][.) -]+[A-Z -]+[:\.\n]') #[1-9]|
    titulosMayusculasEncontrados = []
    i=1
    for m in tituloMayusculas.finditer(text):
        if len(m.group()) > 8 and re.findall(str(i),m.group()):
            titulosMayusculasEncontrados.append(m.group())
    return titulosMayusculasEncontrados
# titulosMAyuscula: lista que guarda los títulos en mayusculas
titulosLatinosMayusculas=[]
for expediente in dfPrueba:
    if len(buscarTitulosLatinosMayusculas(expediente,str(dicTitulos))) > 3:
        titulosLatinosMayusculas.append(buscarTitulosLatinosMayusculas(expediente,dicTitulos))

In [ ]:
titulosLatinosMayusculas

[['1-ANTECEDENTES DE AUTOS DE INTERES MEDICO LEGAL \n',
  '1-ANTECEDENTES FAMILIARES \n',
  '1. EXAMEN PSIQUIATRICO \n',
  '1-FUNDAMENTACION DIAGNOSTICA MEDICA \n',
  '1-EXAMENES COMPLEMENTARIOS SOLICITADOS \n',
  '1-FISICA \n',
  '1- POR LA PARTE ACTORA \n'],
 ['1-ANTECEDENTES DE AUTOS DE INTERES MEDICO LEGAL \n',
  '1-ANTECEDENTES FAMILIARES \n',
  '1. EXAMEN PSIQUIATRICO \n',
  '1-FUNDAMENTACION DIAGNOSTICA MEDICA \n',
  '1-EXAMENES COMPLEMENTARIOS SOLICITADOS \n',
  '1-FISICA \n',
  '1- TRATADO DE ANATOMIA HUMANA L.',
  '1- POR LA PARTE ACTORA \n'],
 ['1- RMN MANO IZQUIERDA \n',
  '1-PUNTOS DE PERICIA PARTE ACTORA \n',
  '1- HERIDA CORTANTE EN DORSO DE MANO IZQUIERDA CON SECUELAS \n',
  '1-DIFICULTAD PARA LA REALIZACION DE LAS TAREAS HABITUALES INTERMEDIA \n'],
 ['1-ANTECEDENTES DE AUTOS DE INTERES MEDICO LEGAL \n',
  '1- ANTECEDENTES FAMILIARES \n',
  '1- EXAMEN FISICO \n',
  '1- DE LA PARTE ACTORA \n'],
 ['1-ANTECEDENTES DE AUTOS DE INTERES MEDICO LEGAL \n',
  '1-ANTECEDENTES FAM